In [7]:
options = ["piedra","tijeras","papel"]

def winner(p1,p2):
    if p1 == p2:
        result = 0
    elif p1 == "piedra" and p2 == "tijeras":
        result = 1
    elif p1 == "piedra" and p2 == "papel":
        result = 2
    elif p1 == "tijeras" and p2 == "piedra":
        result = 2
    elif p1 == "tijeras" and p2 == "papel":
        result = 1
    elif p1 == "papel" and p2 == "piedra":
        result = 2
    elif p1 == "papel" and p2 == "tijeras":
        result = 2
    return result

test = [
    
    ["piedra","piedra", 0],
    ["piedra","tijeras",1],
    ["papel","tijeras",2]
]

for p in test:
    print(p[0]," vs ",p[1],", gana: ",winner(p[0],p[1]))

piedra  vs  piedra , gana:  0
piedra  vs  tijeras , gana:  1
papel  vs  tijeras , gana:  2


In [10]:
from random import choice

def get_choice():
    return choice(options)

for i in range(10):
    player1 = get_choice()
    player2 = get_choice()
    print(player1, player2, winner(player1,player2))

piedra tijeras 1
piedra papel 2
piedra papel 2
piedra papel 2
tijeras papel 1
tijeras papel 1
tijeras papel 1
piedra piedra 0
piedra papel 2
piedra papel 2


In [12]:
def str_to_list(option):
    if option == "piedra":
        res = [1,0,0]
    elif option == "tijeras":
        res = [0,1,0]
    else:
        res = [0,0,1]
    return res

data_x = list(map(str_to_list,["piedra","tijeras","papel"]))
data_y = list(map(str_to_list,["papel","piedra","tijeras"]))

print (data_x)
print (data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [13]:
from sklearn.neural_network import MLPClassifier

In [14]:
clf = MLPClassifier(verbose=False, warm_start=True)


In [16]:
model = clf.fit([data_x[0]],[data_y[0]])
print (model)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=True)


C:\Users\Nikko\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [22]:
def play_n_learn(iters = 10, debug=False):
    score = {"win":0, "lose":0}
    
    data_x = []
    data_y = []
    
    for i in range(iters):
        p1 = get_choice()
        
        pre = model.predict_proba([str_to_list(p1)])[0]
        
        if pre[0] >= 0.95:
            p2 = options[0]
        elif pre[1] >= 0.95:
            p2 = options[1]
        elif pre[2] >= 0.95:
            p2 = options[2]
        else:
            p2 = get_choice()
        
        if debug == True:
            print("p1: ",p1,"p2(red): ",pre," resultado: ",p2)
        
        win = winner(p1,p2)
        
        if debug == True:
            print("p1 vs p2: ", win)
        
        if win == 2:
            data_x.append(str_to_list(p1))
            data_y.append(str_to_list(p2))
            score["win"]+=1
        else:
            score["lose"]+=1
            
    return score,data_x,data_y



In [23]:
sc, dx, dy = play_n_learn(1,debug=True)

p1:  tijeras p2(red):  [0.17097882 0.17212261 0.83668591]  resultado:  piedra
p1 vs p2:  2


In [24]:
print(dx)
print(dy)
print("Score: %s %s %%" % (sc,(sc["win"]*100/(sc["win"]+sc["lose"]))))

if len(dx):
    model = model.partial_fit(dx,dy)

[[0, 1, 0]]
[[1, 0, 0]]
Score: {'win': 1, 'lose': 0} 100.0 %


In [ ]:
# repetir para el train
i = 0
hist_pct = []

while True:
    i+=1
    
    sc,dx,dy = play_n_learn(1000,debug=False)
    pct = (sc["win"]*100/(sc["win"]+sc["lose"]))
    
    hist_pct.append(pct)
    print("Iter %s - score: %s %s %%" % (i,sc,pct))
    
    if len(dx):
        model = model.partial_fit(dx,dy)
    if sum(hist_pct[-9:]) == 900:
        break
    

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show, output_notebook